In [2]:
import numpy as np
import pandas as pd
import os

In [3]:
os.chdir('results/20jul_test/')

In [4]:
os.getcwd()

'/mnt/c/users/isaac/OneDrive/Documents/Isaac/2021-2022 Senior/Economic Research PGC/EconDL/results/20jul_test'

In [12]:
out = np.load('fcast_1_compiled.npz', allow_pickle=True)

In [14]:
out['fcast_cov_mat']

KeyError: 'fcast_cov_mat is not a file in the archive'